In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(                     #input-28x28x1    #RF -3x3  #Output 26x26x16
            nn.Conv2d(1, 16, 3, bias=False),
            nn.ReLU(),          
            nn.BatchNorm2d(16),  
            nn.Dropout2d(0.25),
            nn.Conv2d(16, 16, 3, bias=False),           #input-26x26x16   #RF -5x5  #Output 24x24x16      
            nn.ReLU(), 
            nn.BatchNorm2d(16),          
            nn.Conv2d(16, 16, 3, bias=False),           #input-24x24x16   #RF -7x7  #Output 22x22x16
            nn.ReLU(),         
            nn.BatchNorm2d(16), 
            nn.MaxPool2d(2, 2)                          #input-22x22x16   #RF -7x7  #Output 11x11x16
        )    

        self.conv2 = nn.Sequential(                     #input-11x11x16   #RF -16x16  #Output 9x9x16
            nn.Conv2d(16, 16, 3, bias=False),          
            nn.ReLU(),          
            nn.BatchNorm2d(16),   
            nn.Dropout2d(0.25), 
            nn.Conv2d(16, 16, 3, padding=1, bias=False),  #input-9x9x16   #RF -18x18  #Output 9x9x16   
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
            nn.Conv2d(16, 16, 3,bias=False),             #input-9x9x16   #RF -20x20  #Output 7x7x16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
            nn.Conv2d(16,32, 3, bias=False),             #input-7x7x16   #RF -22x22  #Output 5x5x32
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.25),        
        )

        self.conv_final=nn.Conv2d(32, 10, 1, bias=False) #input-5x5x32 Output: 1x1x32 
        self.avgP = nn.AvgPool2d(5)            


    def forward(self, x):
        x = self.conv1(x)
        #x = self.transition_layer(x)
        x = self.conv2(x)
        x = self.conv_final(x)
        x = self.avgP(x)
        x = x.view(-1, 10)
        
        return F.log_softmax(x, dim=1)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
         Dropout2d-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,304
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
            Conv2d-8           [-1, 16, 22, 22]           2,304
              ReLU-9           [-1, 16, 22, 22]               0
      BatchNorm2d-10           [-1, 16, 22, 22]              32
        MaxPool2d-11           [-1, 16, 11, 11]               0
           Conv2d-12             [-1, 16, 9, 9]           2,304
    

In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    Accuracy = float(100. * correct / len(test_loader.dataset))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        Accuracy))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print('\nepoch ',epoch )
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


epoch  1


loss=0.15949223935604095 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.31it/s]



Test set: Average loss: 0.0650, Accuracy: 9821/10000 (98.21%)


epoch  2


loss=0.10433179885149002 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 29.26it/s]



Test set: Average loss: 0.0374, Accuracy: 9869/10000 (98.69%)


epoch  3


...


Test set: Average loss: 0.0204, Accuracy: 9945/10000 (99.45%)


epoch  19


loss=0.017178891226649284 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.82it/s]



Test set: Average loss: 0.0169, Accuracy: 9952/10000 (99.52%)


epoch  20


loss=0.09224598854780197 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.69it/s]



Test set: Average loss: 0.0192, Accuracy: 9941/10000 (99.41%)

